In [1]:
import typhoon_model.typhoon_model as tm
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

init_notebook_mode(connected=True)

In [2]:
#Define Profject Properties
proj_lat = 14.57
proj_lon = 120.98

year_start = 1989
year_end = 2019

radius = 250 #km

jma_file = r"sample scripts\\bst_all.txt"

ct = tm.Critical_Typhoon(proj_lat, proj_lon, radius)
typ_dict = ct.read_database(jma_file, year_start, year_end, outfreq="1H")

In [5]:
for key, typhoon in typ_dict.items():
    typ_dict[key] = typhoon.round(2)

In [6]:
#For plotting influence area
bearings = np.radians(np.arange(0, 361, 10))
R = 6371

circle = pd.DataFrame(bearings, columns=["bearing"])
circle["lat"] = np.degrees(np.arcsin(np.sin(np.radians(proj_lat)) * np.cos(radius/R) + 
                                     np.cos(np.radians(proj_lat)) * np.sin(radius/R) * np.cos(circle.bearing)))
circle["lon"] = (proj_lon + 
                np.degrees(np.arctan2(np.sin(circle.bearing) * np.sin(radius/R) * np.cos(np.radians(proj_lat)),
                                      np.cos(radius/R) - np.sin(np.radians(proj_lat)) * np.sin(np.radians(circle.lat))))
                )

In [7]:
zoom_level=6
fig = go.Figure(
    layout=dict(
        mapbox=dict(
            style="carto-positron",
            center=go.layout.mapbox.Center(
                lat=proj_lat,
                lon=proj_lon
                ),
            zoom=zoom_level)))

for key, typhoon in typ_dict.items():
    typhoon = typhoon.round(2)
    fig.add_trace(go.Scattermapbox(
        lat=typhoon.lat, lon=typhoon.lon,
        mode="markers+lines", 
        marker=dict(color=typhoon.Vgmax, size=6, colorscale=px.colors.sequential.Jet,
                    colorbar=dict(thickness=20), cmin=10, cmax=70),
        line=dict(color="gray", width=1),
        name=str(typhoon.name[0]) + " " + str(typhoon.year[0])
        )
    )

fig.add_trace(go.Scattermapbox(
        lat=[proj_lat], lon=[proj_lon],
        mode="markers", marker_size=10, marker_color="black", marker_symbol="circle",
        name="Project Site")
             )

fig.add_trace(go.Scattermapbox(
        lat=circle.lat, lon=circle.lon,
        mode="lines", line_color="black",
        name="Project Site")
             )
        
fig.update_layout(showlegend=False, width=800, height=800,
                 title_text="Typhoon Tracks passing within 200km of the Site");

In [8]:
fig.show()

In [9]:
check = True
for key, typhoon in typ_dict.items():
    if check is True:
        typhoons = typhoon
        check = False
    else:
        typhoons = typhoons.append(typhoon)
        
typhoons = typhoons.loc[typhoons["distance"] <= radius].reset_index(drop=True)
typ_vmax = typhoons.loc[typhoons.groupby("year").Vgmax.idxmax()].reset_index(drop=True)
# typ_vmax.sort_values("Vgmax", ascending=False, inplace=True)

typ_vmax["keys"] = typ_vmax.code + " " + typ_vmax.name;

In [10]:
print("Typhoon Tracks with Annual Maximum Wind Speed")
typ_vmax.loc[:, typ_vmax.columns != 'keys']

Typhoon Tracks with Annual Maximum Wind Speed


,date,lat,lon,Pc,Vmax,delP,distance,year,name,code,Vgmax,v_site
0,1989-10-19 04:00:00,16.53,121.87,931.67,50.00,78.33,238.08,1989,Elsie,8927,62.50,21.89
1,1990-06-14 21:00:00,16.25,120.00,1002.00,NaN,8.00,214.32,1990,Nathan,9004,25.57,12.64
2,1991-06-14 08:00:00,14.50,123.23,951.67,43.52,58.33,242.66,1991,Yunya,9105,54.40,20.83
3,1992-07-11 00:00:00,15.70,122.30,965.00,38.89,45.00,189.37,1992,Eli,9205,48.61,23.69
4,1993-12-05 11:00:00,14.23,123.15,959.17,42.13,50.83,236.69,1993,Lola,9326,52.67,21.25
5,1994-10-20 16:00:00,15.27,123.17,958.33,42.60,51.67,247.39,1994,Teresa,9430,53.24,20.61
6,1995-11-02 18:00:00,14.10,123.20,935.00,50.00,75.00,244.81,1995,Angela,9520,62.50,21.76
7,1997-11-15 20:00:00,15.03,123.20,1001.33,NaN,8.67,244.16,1997,Mort,9727,26.92,12.06
8,1998-10-22 00:00:00,14.10,123.20,955.00,41.67,55.00,244.81,1998,Babs,9811,52.09,20.07
9,1999-10-17 05:00:00,15.87,119.38,998.67,NaN,11.33,223.91,1999,Eve,9921,31.99,15.21


In [11]:
zoom_level=6
fig = go.Figure(
    layout=dict(
        mapbox=dict(
            style="carto-positron",
            center=go.layout.mapbox.Center(
                lat=proj_lat,
                lon=proj_lon
                ),
            zoom=zoom_level)))

for key, typhoon in typ_dict.items():
    if key in list(typ_vmax["keys"]):
        fig.add_trace(go.Scattermapbox(
            lat=typhoon.lat, lon=typhoon.lon,
            mode="markers+lines", 
            marker=dict(color=typhoon.Vgmax, size=6, colorscale=px.colors.sequential.Jet,
                        colorbar=dict(thickness=20), cmin=10, cmax=80),
            line=dict(color="gray", width=1),
            name=str(typhoon.name[0]) + " " + str(typhoon.year[0])
            )
        )

fig.add_trace(go.Scattermapbox(
        lat=[proj_lat], lon=[proj_lon],
        mode="markers", marker_size=10, marker_color="black", marker_symbol="circle",
        name="Project Site")
             )

fig.add_trace(go.Scattermapbox(
        lat=circle.lat, lon=circle.lon,
        mode="lines", line_color="black",
        name="Project Site")
             )
        
fig.update_layout(showlegend=False, width=800, height=800,
                 title_text="Typhoon Tracks with Annual Maximum Wind Speed");

In [12]:
fig.show()

In [13]:
check = True
for key, typhoon in typ_dict.items():
    if check is True:
        typhoons = typhoon
        check = False
    else:
        typhoons = typhoons.append(typhoon)
        
typhoons = typhoons.loc[typhoons["distance"] <= radius].reset_index(drop=True)
typ_vmax = typhoons.loc[typhoons.groupby("year").distance.idxmin()].reset_index(drop=True)
typ_vmax.sort_values("distance", ascending=True, inplace=True)

typ_vmax["keys"] = typ_vmax.code + " " + typ_vmax.name;

In [14]:
print("Typhoon Tracks with Annual Minimum Closest Distance")
typ_vmax.loc[:, typ_vmax.columns != 'keys']

Typhoon Tracks with Annual Minimum Closest Distance


,date,lat,lon,Pc,Vmax,delP,distance,year,name,code,Vgmax,v_site
27,2017-09-12 05:00:00,14.57,120.93,1000.33,NaN,9.67,5.04,2017,Doksuri,1719,28.88,0.00
10,2000-11-02 21:00:00,14.65,121.00,994.00,23.61,16.00,9.15,2000,Bebinca,0021,29.52,0.01
6,1995-10-07 23:00:00,14.47,121.00,1008.00,NaN,2.00,11.69,1995,Ted,9516,10.47,0.01
18,2008-06-21 23:00:00,14.67,121.10,974.17,41.67,35.83,16.80,2008,Fengshen,0806,52.09,1.58
9,1999-10-16 17:00:00,14.75,121.03,1004.00,NaN,6.00,20.82,1999,Eve,9921,21.24,0.58
16,2006-09-28 04:00:00,14.37,120.83,973.33,34.26,36.67,27.58,2006,Xangsane,0615,42.83,10.33
20,2010-07-13 17:00:00,14.33,121.13,979.17,33.80,30.83,31.07,2010,Conson,1002,42.25,12.66
5,1994-10-21 07:00:00,14.23,121.00,980.83,32.87,29.17,37.50,1994,Teresa,9430,41.09,19.77
19,2009-10-30 21:00:00,14.15,120.95,987.50,29.17,22.50,46.81,2009,Mirinae,0921,36.46,24.20
3,1992-10-26 01:00:00,14.93,120.75,980.83,30.09,29.17,47.37,1992,Colleen,9226,37.62,27.25


In [15]:
zoom_level=6
fig = go.Figure(
    layout=dict(
        mapbox=dict(
            style="carto-positron",
            center=go.layout.mapbox.Center(
                lat=proj_lat,
                lon=proj_lon
                ),
            zoom=zoom_level)))

for key, typhoon in typ_dict.items():
    if key in list(typ_vmax["keys"]):
        fig.add_trace(go.Scattermapbox(
            lat=typhoon.lat, lon=typhoon.lon,
            mode="markers+lines", 
            marker=dict(color=typhoon.Vgmax, size=6, colorscale=px.colors.sequential.Jet,
                        colorbar=dict(thickness=20), cmin=10, cmax=80),
            line=dict(color="gray", width=1),
            name=str(typhoon.name[0]) + " " + str(typhoon.year[0])
            )
        )

fig.add_trace(go.Scattermapbox(
        lat=[proj_lat], lon=[proj_lon],
        mode="markers", marker_size=10, marker_color="black", marker_symbol="circle",
        name="Project Site")
             )

fig.add_trace(go.Scattermapbox(
        lat=circle.lat, lon=circle.lon,
        mode="lines", line_color="black",
        name="Project Site")
             )

fig.update_layout(showlegend=False, width=800, height=800,
                 title_text="Typhoon Tracks with Annual Minimum Closest Distance");

In [16]:
fig.show()

In [17]:
check = True
for key, typhoon in typ_dict.items():
    if check is True:
        typhoons = typhoon
        check = False
    else:
        typhoons = typhoons.append(typhoon)
        
typhoons = typhoons.loc[typhoons["distance"] <= radius].reset_index(drop=True)
typ_vmax = typhoons.loc[typhoons.groupby("year").Pc.idxmin()].reset_index(drop=True)
# typ_vmax.sort_values("Pc", ascending=True, inplace=True)

typ_vmax["keys"] = typ_vmax.code + " " + typ_vmax.name;

In [18]:
print("Typhoon Tracks with Annual Minimum Central Pressure")
typ_vmax.loc[:, typ_vmax.columns != 'keys']

Typhoon Tracks with Annual Minimum Central Pressure


,date,lat,lon,Pc,Vmax,delP,distance,year,name,code,Vgmax,v_site
0,1989-10-19 04:00:00,16.53,121.87,931.67,50.00,78.33,238.08,1989,Elsie,8927,62.50,21.89
1,1990-06-14 21:00:00,16.25,120.00,1002.00,NaN,8.00,214.32,1990,Nathan,9004,25.57,12.64
2,1991-06-14 08:00:00,14.50,123.23,951.67,43.52,58.33,242.66,1991,Yunya,9105,54.40,20.83
3,1992-07-11 00:00:00,15.70,122.30,965.00,38.89,45.00,189.37,1992,Eli,9205,48.61,23.69
4,1993-11-01 05:00:00,16.08,122.52,959.17,39.35,50.83,235.52,1993,Ira,9323,49.19,19.94
5,1994-10-20 16:00:00,15.27,123.17,958.33,42.60,51.67,247.39,1994,Teresa,9430,53.24,20.61
6,1995-11-02 18:00:00,14.10,123.20,935.00,50.00,75.00,244.81,1995,Angela,9520,62.50,21.76
7,1997-11-15 20:00:00,15.03,123.20,1001.33,NaN,8.67,244.16,1997,Mort,9727,26.92,12.06
8,1998-10-22 00:00:00,14.10,123.20,955.00,41.67,55.00,244.81,1998,Babs,9811,52.09,20.07
9,1999-10-17 06:00:00,16.00,119.20,998.00,19.45,12.00,248.46,1999,Eve,9921,24.31,10.65


In [19]:
zoom_level=6
fig = go.Figure(
    layout=dict(
        mapbox=dict(
            style="carto-positron",
            center=go.layout.mapbox.Center(
                lat=proj_lat,
                lon=proj_lon
                ),
            zoom=zoom_level)))

for key, typhoon in typ_dict.items():
    if key in list(typ_vmax["keys"]):
        fig.add_trace(go.Scattermapbox(
            lat=typhoon.lat, lon=typhoon.lon,
            mode="markers+lines", 
            marker=dict(color=typhoon.Vgmax, size=6, colorscale=px.colors.sequential.Jet,
                        colorbar=dict(thickness=20), cmin=10, cmax=80),
            line=dict(color="gray", width=1),
            name=str(typhoon.name[0]) + " " + str(typhoon.year[0])
            )
        )

fig.add_trace(go.Scattermapbox(
        lat=[proj_lat], lon=[proj_lon],
        mode="markers", marker_size=10, marker_color="black", marker_symbol="circle",
        name="Project Site")
             )

fig.add_trace(go.Scattermapbox(
        lat=circle.lat, lon=circle.lon,
        mode="lines", line_color="black",
        name="Project Site")
             )

fig.update_layout(showlegend=False, width=800, height=800,
                 title_text="Typhoon Tracks with Annual Minimum Central Pressure");

In [20]:
fig.show()

In [21]:
check = True
for key, typhoon in typ_dict.items():
    if check is True:
        typhoons = typhoon
        check = False
    else:
        typhoons = typhoons.append(typhoon)
        
typhoons = typhoons.loc[typhoons["distance"] <= radius].reset_index(drop=True)
typ_vmax = typhoons.loc[typhoons.groupby("year").v_site.idxmax()].reset_index(drop=True)
# typ_vmax.sort_values("Pc", ascending=True, inplace=True)

typ_vmax["keys"] = typ_vmax.code + " " + typ_vmax.name;

In [22]:
print("Typhoon Tracks with Annual Minimum Central Pressure")
typ_vmax.loc[:, typ_vmax.columns != 'keys']

Typhoon Tracks with Annual Minimum Central Pressure


,date,lat,lon,Pc,Vmax,delP,distance,year,name,code,Vgmax,v_site
0,1989-10-10 22:00:00,14.07,120.73,971.67,35.19,38.33,61.96,1989,Dan,8926,43.98,33.99
1,1990-06-14 17:00:00,15.33,121.33,1004.00,NaN,6.00,92.98,1990,Nathan,9004,21.24,15.64
2,1991-06-14 18:00:00,15.00,122.00,965.00,38.89,45.00,119.63,1991,Yunya,9105,48.61,30.29
3,1992-10-25 23:00:00,15.02,121.22,980.00,30.56,30.00,55.81,1992,Colleen,9226,38.20,30.56
4,1993-12-05 22:00:00,13.87,121.47,971.67,35.19,38.33,94.17,1993,Lola,9326,43.98,30.40
5,1994-10-21 04:00:00,14.47,121.53,976.67,35.19,33.33,60.66,1994,Teresa,9430,43.98,34.43
6,1995-11-03 02:00:00,14.27,121.33,951.67,45.37,58.33,50.85,1995,Angela,9520,56.72,43.98
7,1997-05-26 07:00:00,16.30,120.90,1002.00,NaN,8.00,192.56,1997,Levi,9704,25.57,13.57
8,1998-10-22 14:00:00,15.13,121.77,966.67,37.97,43.33,105.23,1998,Babs,9811,47.46,31.23
9,1999-10-16 22:00:00,15.07,120.50,1002.67,NaN,7.33,75.58,1999,Eve,9921,24.17,18.79


In [25]:
zoom_level=6
fig = go.Figure(
    layout=dict(
        mapbox=dict(
            style="carto-positron",
            center=go.layout.mapbox.Center(
                lat=proj_lat,
                lon=proj_lon
                ),
            zoom=zoom_level)))

for key, typhoon in typ_dict.items():
    if key in list(typ_vmax["keys"]):
        fig.add_trace(go.Scattermapbox(
            lat=typhoon.lat, lon=typhoon.lon,
            mode="markers+lines", 
            marker=dict(color=typhoon.v_site, size=6, colorscale=px.colors.sequential.Jet,
                        colorbar=dict(thickness=20), cmin=0, cmax=50),
            line=dict(color="gray", width=1),
            name=str(typhoon.name[0]) + " " + str(typhoon.year[0])
            )
        )

fig.add_trace(go.Scattermapbox(
        lat=[proj_lat], lon=[proj_lon],
        mode="markers", marker_size=10, marker_color="black", marker_symbol="circle",
        name="Project Site")
             )

fig.add_trace(go.Scattermapbox(
        lat=circle.lat, lon=circle.lon,
        mode="lines", line_color="black",
        name="Project Site")
             )

fig.update_layout(showlegend=False, width=800, height=800,
                 title_text="Typhoon Tracks with Annual Maximum Estimated Site Wind Speed");

In [26]:
fig.show()